In [1]:
import sys, pathlib, os, pickle, ast
import numpy as np
from tqdm import tqdm
from func_timeout import func_timeout, FunctionTimedOut

# Parameters

In [2]:
# Provide your OpenAI API key here
os.environ["OPENAI_API_KEY"] = ""

# Load problem data

In [3]:
problem_folder = '../problems/WSCP_time_dependent'
problem_data_file = f"{problem_folder}/problem_data.pkl"

In [4]:
project_root = pathlib.Path().resolve().parent
sys.path.insert(0, str(project_root))
from utils.OptiHive import OptiHive
from utils.generation import generate_solvers, generate_instance, generate_validity_tests, execute_code, compute_outputs, run_validity_tests

problem_path = pathlib.Path().resolve() / problem_folder
sys.path.insert(0, str(problem_path))
from feasibility import evaluate_feasibility

In [5]:
if os.path.exists(problem_data_file):
    with open(problem_data_file, 'rb') as file:
        problem_data = pickle.load(file)
else:
    problem_data = {'name': problem_folder.split('/')[-1]}
    with open(f"{problem_folder}/problem_specification/description.txt", 'r') as file:
        problem_data['problem_description'] = file.read()
    with open(f"{problem_folder}/problem_specification/input_template.txt", 'r') as file:
        problem_data['input_template'] = file.read()
    with open(f"{problem_folder}/problem_specification/output_template.txt", 'r') as file:
        problem_data['output_template'] = file.read()
    with open(f"{problem_folder}/problem_specification/minimization.txt", 'r') as file:
        problem_data['minimization'] = ast.literal_eval(file.read())

problem_data['evaluate_feasibility'] = evaluate_feasibility 

with open(problem_data_file, 'wb') as file:
    pickle.dump(problem_data, file)

# Generate, evaluate and save components

In [ ]:
optiHive = OptiHive(
    problem_folder, 
    solver_model = "gpt-4.1-mini", 
    instance_model = "gpt-4.1-mini", 
    test_model = "gpt-4.1-mini", 
    SOLVERS_BATCH_SIZE = 50
)

In [ ]:
n_solvers = 100
n_instances = 100
n_validity_tests = 100

optiHive.generate_components(n_solvers, n_instances, n_validity_tests)
optiHive.evaluate(verbose=True);